# Import necessary libraries

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pickle
from multiprocessing import Pool
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import *

from sklearn.ensemble import *

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import *
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score

# Data import

In [2]:
import os, sys
os.chdir("/home/sieu/PycharmProjects/ML-Vincent-Ng/project")

## File to Pandas Dataframe

Add column names into columns

In [3]:
column_name = []
# Open the file with relative path
with open('./attr.txt', 'r') as infile:
    # Read all data lines as a list of strings
    data_lines = infile.readlines() 

    # Extract each data line
    for ind, line in enumerate(data_lines):
        line_noendl = line[:-1] # remove the last character of string, which is '\n'
        data = line_noendl.split(':') # split the string by tab '\t' to get a tuple 
        column_name.append(data[0])

Import data to DataFrame

In [4]:
# For experiment
# take 10 minutes
# Should create a new txt file instead for replace ' ' with '\t'
df2 = pd.DataFrame(columns = column_name)
# Open the file with relative path
with open('./train.txt', 'r') as infile:
    # Read all data lines as a list of strings
    data_lines = infile.readlines() 
    with open('./train2.txt', 'w', encoding = 'utf-8') as outfile:
      for column in column_name:
        outfile.write(column)
        outfile.write('\t')
      outfile.write('\n')
      # Extract each data line
      for ind, line in enumerate(data_lines):
        line_noendl = line[:-1] # remove the last character of string, which is '\n'
        data = line_noendl.split('\t') # split the string by tab '\t' to get a tuple 
        append_list = []
        for i in data:
          a = i.split(' ') # additional for removing the ' ' in the dataset
          for j in a:
            if (j!=''):
              append_list.append(j)
        for idx, value in enumerate(append_list):
          outfile.write(value)
          if (idx<len(column_name)-1):
            outfile.write('\t')
        outfile.write('\n')
df2 = pd.read_table('train2.txt', sep = '\t')
df = df2.iloc[:,:-1]

In [5]:
df = df.drop_duplicates()
df = df.dropna()
# Save year
X_year =  df.loc[:,'YEAR']
df.drop(['YEAR'], axis=1)

,B1,B2,B3,C1,C2,C3,C4,C5,C6,C7,...,CT22,CT23,CT24,CT25,CT26,CH1,CH2,CH3,CH4,Class
0,0,0,1,1.260,1.17,0.720,4.590,0.45,0.765,0.540,...,3,2,2,1,3,0,0,0,0,1
1,0,1,0,0.450,0.81,0.000,0.000,0.00,0.855,0.000,...,3,2,1,1,3,0,0,0,0,1
2,0,0,1,0.540,2.88,0.000,0.000,0.00,0.765,0.000,...,3,2,2,1,3,0,0,0,0,1
3,0,0,1,0.810,1.35,0.450,0.000,0.00,0.000,0.720,...,3,2,2,1,3,0,0,0,0,1
4,0,0,1,0.900,1.17,0.765,0.000,0.00,0.630,0.810,...,4,3,2,1,3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0,1,0,0.450,1.17,0.630,1.215,0.45,0.990,0.540,...,3,2,2,1,3,0,0,0,0,0
11996,0,1,0,5.085,0.90,0.000,0.000,0.00,0.000,0.945,...,4,3,2,1,3,0,0,0,0,1
11997,0,0,1,1.170,1.71,0.630,0.000,0.00,0.720,0.000,...,3,2,1,1,3,0,0,0,0,0
11998,0,0,1,0.810,1.17,0.000,0.000,0.00,0.990,0.810,...,5,3,2,2,4,0,0,0,0,1


In [6]:
# Seperate data by type
y = df["Class"]
X_B = df.loc[:, 'B1':'B3']
X_C = df.loc[:,'C1':'C139']
X_CT = df.loc[:,'CT1':'CT26']
X_H = df.loc[:,'CH1':'CH4']

In [7]:
# Calculate statistics
print(X_B.describe())
print(X_C.describe())
print(X_CT.describe())
print(X_H.describe())
print(X_year.describe())

                 B1            B2            B3
count  11982.000000  11982.000000  11982.000000
mean       0.332916      0.340427      0.326657
std        0.471276      0.473873      0.469010
min        0.000000      0.000000      0.000000
25%        0.000000      0.000000      0.000000
50%        0.000000      0.000000      0.000000
75%        1.000000      1.000000      1.000000
max        1.000000      1.000000      1.000000
                 C1            C2            C3            C4            C5  \
count  11982.000000  11982.000000  11982.000000  11982.000000  11982.000000   
mean       2.087392      9.599654      0.625287      0.287599      0.148434   
std       13.490473     32.454435      1.112775      0.536472      0.405063   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.450000      1.080000      0.000000      0.000000      0.000000   
50%        0.720000      1.350000      0.630000      0.000000      0.000000   
75%        1.17

In [8]:
# Convert to numpy array
y = y.to_numpy()
X_B = X_B.to_numpy()
X_C = X_C.to_numpy()
X_CT = X_CT.to_numpy()
X_H = X_H.to_numpy()
X_year = X_year.to_numpy()

In [9]:
# Set variance threshold for every type of data
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_C = sel.fit_transform(X_C)
X_CT = sel.fit_transform(X_CT)
X_H = sel.fit_transform(X_H)
print(X_C.shape)
print(X_CT.shape)
print(X_H.shape)
print("Done variance selection")
X=np.concatenate((X_C, X_CT, X_H), axis=1)
print(X.shape)
selX = SequentialFeatureSelector(estimator=RidgeClassifierCV(), n_features_to_select=X.shape[1]//10, direction="forward")
X = selX.fit_transform(X, y)
print('Perform PCA on remaining feature to filter noises')
pca = PCA(svd_solver='full', n_components='mle')
X = pca.fit_transform(X)
print(X.shape)

(11982, 99)
(11982, 18)
(11982, 2)
Done variance selection
(11982, 119)
Perform PCA on remaining feature to filter noises
(11982, 10)


In [10]:
# Concat X_B with X to get the final attribute array
X_new=np.concatenate((X_B,X), axis=1)

# GridSearch for best model

In [211]:
#params = {'max_iter':np.arange(200,600,100),'max_leaf_nodes':np.arange(700,1000, 10), 'l2_regularization':np.arange(0.5,1,0.05), 'learning_rate':np.arange(0.5,1,0.05)}
params = {'max_iter':np.arange(50,200,100),'max_leaf_nodes':np.arange(100,500, 10), 'l2_regularization':np.arange(0,1,0.05)}
search = HalvingRandomSearchCV(estimator=HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), 
                                param_distributions=params, factor=2,
                                n_jobs=-1).fit(X_new,y)

In [212]:
# Best model validation
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
arr = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2], max_leaf_nodes=800,max_iter=300,learning_rate=0.8500000000000002,l2_regularization=0.8500000000000001).fit(X_train, y_train)
    #print(clf.score(X_test, y_test))
    #arr.append(clf.score(X_test, y_test))
    arr.append(search.score(X_test, y_test))
    print(search.score(X_test, y_test))
print("Done")
print("mean acc: ", np.mean(arr))

0.8640533778148457
0.854045037531276
0.8564273789649416
0.8597662771285476
0.8606010016694491
0.8564273789649416
0.8664440734557596
0.8689482470784641
0.8564273789649416
0.8722871452420701
Done
mean acc:  0.8615427296815238


In [ ]:
# Best model list
model = []

In [ ]:
# Adding best model to list
model.append(search)

In [ ]:
# Remove model from list
model.pop()

In [126]:
# Save models
with open("92_model.pkl", 'wb') as file:
    pickle.dump(search, file)

In [206]:
# Load model
with open("77_model.pkl", 'rb') as file:
    search = pickle.load(file)

In [207]:
search.best_params_

{'max_leaf_nodes': 650,
 'max_iter': 200,
 'max_depth': 6,
 'l2_regularization': 0.8500000000000001}

# Essemble

In [ ]:
def Ridge_classifiers(data):
    clf = RidgeClassifier(solver="svd")
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def knn_classifiers(data):
    clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def svc_classifiers(data):
    clf = SVC(kernel='rbf', degree=3)
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def BernoulliNB_classifiers(data):
    clf = BernoulliNB()
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def HGB_classifiers(data):
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2])
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)

In [ ]:
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
dataset = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dataset.append((X_train, y_train, X_test, y_test))
    #Testing individual clf
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2], max_leaf_nodes=800,max_iter=300,learning_rate=0.8500000000000002,l2_regularization=0.8500000000000001).fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = RidgeClassifier(solver="svd").fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = BernoulliNB().fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
print("Done")

In [14]:
assembly = []
vote_weights = []
def append_results(result):
    for clf, score in result:
        assembly.append(clf)
        vote_weights.append(score)
if __name__ == '__main__':
    p = Pool()
    p.map_async(Ridge_classifiers, dataset, callback=append_results)
    #p.map_async(nn_classifiers, dataset, callback=append_results)
    #p.map_async(knn_classifiers, dataset, callback=append_results)
    #p.map_async(svc_classifiers, dataset, callback=append_results)
    p.map_async(BernoulliNB_classifiers, dataset, callback=append_results)
    p.map_async(HGB_classifiers, dataset, callback=append_results)
    p.close()
    p.join()

Process ForkPoolWorker-81:
Process ForkPoolWorker-59:
Process ForkPoolWorker-72:
Process ForkPoolWorker-85:
Process ForkPoolWorker-52:
Process ForkPoolWorker-55:
Process ForkPoolWorker-58:
Process ForkPoolWorker-34:
Process ForkPoolWorker-39:
Process ForkPoolWorker-68:
Process ForkPoolWorker-84:
Process ForkPoolWorker-71:
Process ForkPoolWorker-87:
Process ForkPoolWorker-82:
Process ForkPoolWorker-88:
Process ForkPoolWorker-76:
Process ForkPoolWorker-65:


KeyboardInterrupt: 

Process ForkPoolWorker-80:
Process ForkPoolWorker-63:
Process ForkPoolWorker-61:
Process ForkPoolWorker-73:
Process ForkPoolWorker-66:
Process ForkPoolWorker-74:
Process ForkPoolWorker-83:
Process ForkPoolWorker-62:
Process ForkPoolWorker-78:
Process ForkPoolWorker-70:
Process ForkPoolWorker-53:
Process ForkPoolWorker-77:
Process ForkPoolWorker-79:
Process ForkPoolWorker-86:
Process ForkPoolWorker-57:
Process ForkPoolWorker-69:
Process ForkPoolWorker-60:
Process ForkPoolWorker-64:
Process ForkPoolWorker-67:
Process ForkPoolWorker-47:
Process ForkPoolWorker-75:
Process ForkPoolWorker-56:
Process ForkPoolWorker-50:
Process ForkPoolWorker-36:
Process ForkPoolWorker-35:
Process ForkPoolWorker-44:
Process ForkPoolWorker-48:
Process ForkPoolWorker-43:
Process ForkPoolWorker-49:
Process ForkPoolWorker-40:
Process ForkPoolWorker-54:
Process ForkPoolWorker-41:
Process ForkPoolWorker-32:
Process ForkPoolWorker-37:
Process ForkPoolWorker-45:
Process ForkPoolWorker-46:
Process ForkPoolWorker-42:
P

  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self

  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in w

  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/p

  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 364, in get
    with self._rlock:
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/anaconda3/envs/

  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/queues.py", line 365, in get
    res = self._reader.recv_bytes()
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/connection.py", line 221, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/connection.py", line 419, in _recv_bytes
    buf = self._recv(4)
  File "/opt/anaconda3/envs/AIC21_MTMC/lib/python3.10/multiprocessing/connection.py", line 384, in _recv
    chunk = read(handle, remaining)
KeyboardInterrup

In [30]:
print(assembly)
print(vote_weights)

[HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), HistGradientBoostingClassifier(categorical_features=[0, 1, 2])]
[0.7297748123436196, 0.7272727272727273, 0.7078464106844741, 0.7212020033388982, 0.7170283806343907, 0.7387312186978298, 0.7053422370617696, 0.7036727879799666, 0.7086811352253757, 0.7203672787979967]


In [14]:
clfs = []
for i in range(len(assembly)):
    clfs.append(("Classifier " + str(i), assembly[i]))
eclf = VotingClassifier(estimators=clfs,voting='hard', weights=vote_weights, n_jobs=-1)

In [26]:
cross_val_score(eclf, X_new, y, scoring='accuracy', cv=10, n_jobs=-1)

array([0.69307756, 0.71142619, 0.71869783, 0.70367279, 0.71702838,
       0.71869783, 0.70283806, 0.70367279, 0.68196995, 0.67529215])

# K-Fold 